In [1]:
import tkinter as tk
from tkinter import filedialog
import os
import pandas as pd
from  matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import numpy as np
import seaborn as snb; snb.set();

In [2]:
class Tablas:
    def __init__(self, path):
        self.path = path
        self.dict_dict = {}
        self.Day_BrakDown = ["Day_BreakDown_(%)_Night-Clear-Cloudy-Suspect",
                             "Day_BreakDown_(%)_Parked-Passive-Active-Event",
                             "Day_BreakDown_(min)_Cleaning-ADtest-Other-Bad"]

   
    def crear_diccionarios(self):
        '''Se crea un dicionario de diccionarios con todos los datos'''
        with open(self.path) as archivo:
            for linea in archivo.readlines():
                
                # Omitimos los salto de linea
                if linea.strip() == "" or linea[0]==";":
                    continue
                    
                # Creamos el nombre del diccionario.
                if linea[0] == "[":
                    # Creamos el título para cada tabla 
                    titulo_dict = linea[1:linea.find("]", 1)].replace(" ", "_")
                    # Creamos un diccionario con el título
                    self.dict_dict[titulo_dict] = {}
                    continue
                    
                # Agregamos los datos al diccionario que se está leyendo
                llave = str(linea[0:linea.find("=")])
                valor = str([linea[linea.find("=")+1:linea.find("\\")]])
                valor = str(valor).replace("['","").replace("']","").strip()
                valor = valor.split(",")
                
                valor = self.Pasar_a_int(valor)
                
                if titulo_dict in self.Day_BrakDown:
                    valor = self.Pasar_a_listas(valor)
                
                
                    
                self.dict_dict[titulo_dict][llave] = valor      
                
        return self.dict_dict
    
    
    def enlistar_diccionarios(self):
        j=1
        for i in self.dict_dict:
            print(f"{j}.- {i}")
            j=j+1
            
    def Pasar_a_int(self,valor):
        # Transformamos, en donde es posible, los strings a int
        for i in range(len(valor)):
            try:
                valor[i]=int(valor[i])
            except:
                pass
        return valor
    
    def Pasar_a_listas(self,valor):
        # Transformamos, en donde es posible, los strings a listas
        for i in range(len(valor)):
            try:
                valor[i]=valor[i].split("-")
                # Convertimos los elementos de la lista que sean posibles de convertir a flotante
                for j in range(len(valor[i])):
                    try:
                        valor[i][j]=valor[i][j].strip()
                        valor[i][j]=float(valor[i][j])
                    except:                        
                        pass
            except:
                pass
            
        return valor
    
    def acomodar_datos(self,path):
        # Creamos las llaves a partir del titulo de la tabla 
        llaves = path.split("-")
        l0 = llaves[0].split("_")[0]
        l1 = llaves[0].split("_")[-1]
        llaves[0] = l1
        llaves.insert(0,l0)
        diccionario = {}
        
        # Acomodamos los datos en diccionarios que podamos transformar a dataframes posteriormente
        for key,values in self.dict_dict.get(path).items():
            
            
            if len(list(diccionario.keys()))==0:
                for i in range(len(values)):
                    if (values[i][0])!="Missing":
                        diccionario[(values[i][0])] = {j: [] for j in llaves}
                        
            aux= list(diccionario.keys())
            #print(values[0])
            
            for t,k in enumerate(aux):
                diccionario[k][llaves[0]].append(f"{key[0:3]}  {key[3:]}")
                for l in [1,2,3,4]:
                    diccionario[k][llaves[l]].append(values[t][l])

        self.dict_dict[path]=diccionario
        
    def crear_tablas(self):
        Tablas = self.crear_diccionarios()
        for i in self.Day_BrakDown:
            self.acomodar_datos(i)
        return Tablas

In [3]:
def graficas(Path,Grafica):
    y_s=[]
    Nombre_Grafica = Path.replace("_"," ").replace("-", " - ")
    Nombre_Grafica = Nombre_Grafica[:Nombre_Grafica.find("%)")+2]
    Tablas_Totales = DataFrame[Path]
    Nombres_Tablas = list(Tablas_Totales.keys())

    Tablas_Individuales = DataFrame[Path][Nombres_Tablas[Grafica]]
    Nombres_Columnas = list(Tablas_Individuales)

    x_s = np.array(Tablas_Individuales[Nombres_Columnas[0]])
    for i in range(1,len(Nombres_Columnas)):
        y_s.append(np.array(Tablas_Individuales[Nombres_Columnas[i]]))
    
    
    for i in range(len(x_s)):
        x_s[i] = x_s[i][5:]
    x_s=x_s.astype(int)   

    fig,axs = plt.subplots(1,1,figsize=(13,8));

    
    for i in range(0,len(Nombres_Columnas)-1):
        axs = plt.plot(x_s,y_s[i],label=f"{Nombres_Columnas[i+1]}")
        
    plt.title(f"{Nombre_Grafica}: {Nombres_Tablas[Grafica]}",fontsize=30)
    plt.legend(fontsize = "x-large")
    plt.xlabel("Dias",fontsize=20)
    plt.ylabel("Porcentaje",fontsize=20);
    plt.close()
    return fig,axs

In [4]:
root = tk.Tk()
root.minsize(width=860,height=200)

lista_archivos = []
Archivo = tk.StringVar()
Archivo.set(" ")
ruta = ""
value=""
value_2=""
Nombre_DataFrame=""

#Creamos un boton para buscar la carpeta con los archivos
boton1 = tk.Button(root, text="Seleccionar carpeta", command=lambda:abrir_carpeta(lista_archivos))
boton1.grid(row=1)

#Configuramos un boton que creara la instancia de las tablas
boton2 = tk.Button(root,text="Seleccionar Archivo",command=lambda: crear_instancia())
boton2.grid(row=1,column=2)
boton2.configure(state=tk.DISABLED)

#Configuramos un boton que creara la instancia de las tablas
boton3 = tk.Button(root,text="Seleccionar Datos",command=lambda: presentar_grafica())
boton3.grid(row=1,column=3)
boton3.configure(state=tk.DISABLED)

# Lista para mostar los archivos
lista = tk.Listbox(root,width=60)
lista.grid(row=2,column=2,rowspan=10,columnspan=1)

#Lista para mostrar las graficas
lista2 = tk.Listbox(root,width=60)
lista2.grid(row=2,column=3,rowspan=10,columnspan=1)

#Define una función para abrir el diálogo de selección de carpeta
def abrir_carpeta(lista_archivos):
    # Abre el diálogo y guarda la ruta de la carpeta seleccionada
    global path
    path = filedialog.askdirectory()
    boton1.configure(state=tk.DISABLED)
    if path:
        # Muestra la ruta de la carpeta seleccionada
        lista = os.listdir(path)
        for i in range(len(lista)):
            if lista[i].endswith("report"):
                lista_archivos.append(lista[i])

        for opcion,arch in enumerate(lista_archivos):
            tk.Radiobutton(root,
                        text=arch,
                        value=arch,
                        variable=Archivo,
                        command=lambda: crear_directorio(path),
                        padx=5,
                        pady=5).grid(row=opcion+2,sticky="w")
        else:
            pass
        
def crear_directorio(path):
    global ruta
    if Archivo.get() != " ":
        boton2.configure(state=tk.ACTIVE)
    path=path.replace("/","\\")
    ruta = (f"{path}\{Archivo.get()}") 
    
                      
def crear_instancia():    
    global Diccionarios
    global DataFrame
    
    
    Diccionarios = Tablas(ruta)
    DataFrame = Diccionarios.crear_tablas()
    lista = crear_lista()
                      
                      
def crear_lista():
# Creamos una lista con las posibles opciones
    lista.delete(0,tk.END)
    lista.insert(tk.END,)
    titulo = Archivo.get()
    titulo = "Reporte: " + titulo[0:5] + "  " + titulo[5:8]
    
    label1 = tk.Label(root, text=f"{titulo}").grid(column=2,row=2)
    for i in Diccionarios.Day_BrakDown:
        lista.insert(tk.END, i.replace("_"," ").replace("-"," - "))
        
    lista.bind("<<ListboxSelect>>", seleccionar_elemento)

    return lista

def crear_lista_graficas():
# Creamos una lista con las posibles opciones
    global value
    global Nombre_DataFrame
    index = lista.curselection()
    if index:
        value = lista.get(index[0])
        
    Nombre_DataFrame = value.replace(" - ","-").replace(" ","_")
    lista_aux = list(DataFrame[Nombre_DataFrame].keys())
    
    lista2.delete(0,tk.END)
    
    for i in lista_aux:
        lista2.insert(tk.END,i)
    lista2.bind("<<ListboxSelect>>", mostrar_grafica)
    
    
    return lista2
                      
def seleccionar_elemento(event):
    crear_lista_graficas()
    
def seleccionar_elemento_2(event):
    boton3.config(state=tk.NORMAL)

def mostrar_grafica(event):
    global Nombre_DataFrame
    global value_2
    global fig
    aux=[]
    
    index_2 = lista2.curselection()
    if index_2:
        value_2 = lista2.get(index_2[0])
        boton3.config(state=tk.NORMAL)
        
    for t,l in enumerate(list(DataFrame[Nombre_DataFrame].keys())):
        if l==value_2:
            fig,ax = graficas(Nombre_DataFrame,t)


def presentar_grafica():
    global fig
    top = tk.Toplevel()
    top.title("Grafica")
    try:
        top.deiconify()
        canvas = canvas = FigureCanvasTkAgg(fig, master=top)
        canvas.draw()
        canvas.get_tk_widget().grid(column=0,row=0)
    except:
        pass
        
#Muestra la ventana
root.mainloop()